<a href="https://colab.research.google.com/github/gondow/rust-future/blob/main/notebooks/rust_future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

Rust環境をセットアップする（ちょっと時間かかります）
`export PATH=$HOME/.cargo/bin:$PATH`が効かないのでPythonでやってます

In [3]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:" + os.environ['PATH']
!rustc --version

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-06-26, rust version 1.88.0 (6b00bc388 2025-06-23)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
  9.5 MiB /   9.5 MiB (100 %)   9.4 MiB/s in  1s
info: installing component 'clippy'
info: installing component 'rust-docs'
 20.1 MiB /  20.1 MiB (100 %)   2.9 MiB/s in  7s
info: installing component 'rust-std'
 29.5 MiB /  29.5 MiB (100 %)   8.6 MiB/s in  3s
info: installing component 'rustc'
 76.3 MiB /  76.3 MiB (100 %)   8.4 MiB/s in 10s
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu

ファイル `hello.rs` を作成する

In [ ]:
%%writefile hello.rs
fn main() {
    println!("Hello, world!");
}

Writing hello.rs


`hello.rs` をコンパイルして実行する

In [ ]:
!rustc hello.rs
!./hello

Hello, world!


# Iterator

In [5]:
%%writefile iterator.rs
fn main() {
    for i in 0..3 {
        println!("{}", i);
    }
}

Writing iterator.rs


In [6]:
!rustc iterator.rs
!./iterator

0
1
2


`0..3`はイテレータ．イテレータは`next()`を呼ぶと「次の値」をオプショナル型として返す．

In [1]:
%%writefile generator2.rs
fn main() {
   let mut iter = 0..3;
   println!("{:?}", iter.next()); // Some(0)
   println!("{:?}", iter.next()); // Some(1)
   println!("{:?}", iter.next()); // Some(2)
   println!("{:?}", iter.next()); // None
}

Writing generator2.rs


In [4]:
!rustc generator2.rs
!./generator2

Some(0)
Some(1)
Some(2)
None


自分でもイテレータを定義できる．以下はCounterイテレータの定義例．途中でreturnして値を返して，次にnext()を呼ばれる時，イテレータ構造体の`count`の値が状態として保存されているので，順番にインクリメントされた値が返る．

In [9]:
%%writefile counter.rs
struct Counter {
    count: usize,
}
impl Iterator for Counter {
    type Item = usize;
    fn next(&mut self) -> Option<usize> {
        if self.count < 3 {
            self.count += 1;
            Some(self.count)
        } else {
            None
        }
    }
}
fn main() {
    let mut counter = Counter {count: 0};
    println!("{:?}", counter.next ());
    println!("{:?}", counter.next ());
    println!("{:?}", counter.next ());
    println!("{:?}", counter.next ());
}

Overwriting counter.rs


In [8]:
!rustc counter.rs
!./counter

Some(1)
Some(2)
Some(3)
None


Iteratorの定義は↓こうなっている．
```
pub trait Iterator {
    type Item;
    fn next(&mut self) -> Option<Self::Item>;
    // 他にもたくさん
}
```
詳しくは[公式の説明](https://doc.rust-lang.org/std/iter/trait.Iterator.html)を見てね．

# Generator

In [ ]:
%%writefile generator.rs
#![feature(generators, generator_trait)]
use std::ops::{Generator, GeneratorState};
use std::pin::Pin;
fn main() {
    let mut gen = || {
        yield 1;
        yield 2;
        return 3;
    };

    let mut gen = Pin::new(&mut gen);

    println!("{:?}", gen.resume(())); // yield 1
    println!("{:?}", gen.resume(())); // yield 2
    println!("{:?}", gen.resume(())); // return 3
}

Writing generator.rs


In [ ]:
!rustc generator.rs
!./generator

error[E0557]: feature has been removed
 --> generator.rs:1:12
  |
1 | #![feature(generators, generator_trait)]
  |            ^^^^^^^^^^ feature has been removed
  |
  = note: renamed to `coroutines`

error[E0432]: unresolved imports `std::ops::Generator`, `std::ops::GeneratorState`
 --> generator.rs:2:16
  |
2 | use std::ops::{Generator, GeneratorState};
  |                ^^^^^^^^^  ^^^^^^^^^^^^^^ no `GeneratorState` in `ops`
  |                |
  |                no `Generator` in `ops`

error[E0554]: `#![feature]` may not be used on the stable release channel
 --> generator.rs:1:1
  |
1 | #![feature(generators, generator_trait)]
  | ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

error[E0658]: yield syntax is experimental
 --> generator.rs:6:9
  |
6 |         yield 1;
  |         ^^^^^^^
  |
  = note: see issue #43122 <https://github.com/rust-lang/rust/issues/43122> for more information

error[E0658]: yield syntax is experimental
 --> generator.rs:7:9
  |
7 |         yield 2;
  |       

# Future